In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.3 MB/s eta 0:00:00


In [ ]:
from ultralytics import RTDETR
import cv2

# Load the pretrained RT-DETR-l model
model = RTDETR("rtdetr-l.pt")

# Path to your crowded image
image_path = "/content/Screenshot 2025-12-22 201904.png"

# Run inference
results = model(image_path)
results = results[0]  # get the first result

# Load image with OpenCV
img = cv2.imread(image_path)

# Iterate over boxes and draw only people
for box in results.boxes:
    class_id = int(box.cls[0])
    class_name = results.names[class_id]

    if class_name == "person":
        x_min, y_min, x_max, y_max = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0])

        # Draw rectangle and label
        cv2.rectangle(img, (int(x_min), int(y_min)), (int(x_max), int(y_max)), (0, 255, 0), 2)
        cv2.putText(img, f"{class_name} {conf:.2f}", (int(x_min), int(y_min)-5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

# Save the image with only people boxes
cv2.imwrite("/content/crowd_people_only.png", img)
print("Image saved with boxes only around people!")



image 1/1 /content/Screenshot 2025-12-22 201904.png: 640x640 38 persons, 1 backpack, 21 handbags, 2 suitcases, 1 chair, 5 potted plants, 2455.9ms
Speed: 7.6ms preprocess, 2455.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Image saved with boxes only around people!


In [ ]:
import cv2
import numpy as np
from ultralytics import RTDETR
from shapely.geometry import Point, Polygon
import csv
import os

# --------------------------------
# 1. Load RT-DETR model
# --------------------------------
model = RTDETR("rtdetr-l.pt")

# --------------------------------
# 2. Open video
# --------------------------------
cap = cv2.VideoCapture("ok.mp4")
assert cap.isOpened(), "Error opening video"

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_interval = fps * 3  # every 3 seconds

# --------------------------------
# 3. Get frame size
# --------------------------------
success, frame = cap.read()
h, w, _ = frame.shape
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# --------------------------------
# 4. Define polygon zones
# --------------------------------
zones = {
    "Zone A": Polygon([(0,0), (w//2,0), (w//2,h//3), (0,h//3)]),
    "Zone B": Polygon([(w//2,0), (w,0), (w,h//3), (w//2,h//3)]),
    "Zone C": Polygon([(0,h//3), (w,h//3), (w,2*h//3), (0,2*h//3)]),
    "Zone D": Polygon([(0,2*h//3), (w//2,2*h//3), (w//2,h), (0,h)]),
    "Zone E": Polygon([(w//2,2*h//3), (w,2*h//3), (w,h), (w//2,h)])
}

zone_colors = {
    "Zone A": (0,0,255),
    "Zone B": (255,0,0),
    "Zone C": (0,255,0),
    "Zone D": (0,255,255),
    "Zone E": (255,0,255)
}

# --------------------------------
# 5. Prepare CSV file
# --------------------------------
csv_file = "zone_c.csv"
os.makedirs("frames", exist_ok=True)

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Time_sec", "Zone A", "Zone B", "Zone C", "Zone D", "Zone E", "Frame_Image"])

# --------------------------------
# 6. Process video
# --------------------------------
frame_id = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    if frame_id % frame_interval != 0:
        frame_id += 1
        continue

    zone_counts = {z: 0 for z in zones}

    # RT-DETR inference (person only)
    results = model(frame, conf=0.2, classes=[0])
    detections = results[0]

    for box in detections.boxes:
        class_id = int(box.cls[0])
        class_name = detections.names[class_id]

        if class_name == "person":
            # Bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name} {float(box.conf[0]):.2f}",
                        (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

            # Foot point for zone counting
            foot_x = (x1 + x2) // 2
            foot_y = y2  # bottom of bounding box
            point = Point(foot_x, foot_y)

            for zone_name, poly in zones.items():
                if poly.contains(point):
                    zone_counts[zone_name] += 1
                    cv2.circle(frame, (foot_x, foot_y), 6, zone_colors[zone_name], -1)
                    break

    # Draw zones and counts
    for zone_name, poly in zones.items():
        pts = np.array([(int(x), int(y)) for x, y in poly.exterior.coords])
        cv2.polylines(frame, [pts], True, (0,255,0), 2)
        cv2.putText(frame,
                    f"{zone_name}: {zone_counts[zone_name]}",
                    (pts[0][0] + 5, pts[0][1] + 25),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (255,255,255), 2)

    # Save frame image
    time_sec = frame_id // fps
    frame_name = f"frames/frame_{time_sec}.jpg"
    cv2.imwrite(frame_name, frame)

    # Write to CSV
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            time_sec,
            zone_counts["Zone A"],
            zone_counts["Zone B"],
            zone_counts["Zone C"],
            zone_counts["Zone D"],
            zone_counts["Zone E"],
            frame_name
        ])

    print(f"Saved frame at {time_sec} sec with counts {zone_counts}")

    frame_id += 1

cap.release()
print("Processing done!")

AssertionError: Error opening video

In [ ]:
pip install rfdetr


In [ ]:
import cv2
import numpy as np
from ultralytics import RTDETR
from shapely.geometry import Point, Polygon
import csv
import os

# --------------------------------
# 1. Load RT-DETR model
# --------------------------------

from rfdetr import RFDETRBase

model = RFDETRBase()

# --------------------------------
# 2. Open video
# --------------------------------
cap = cv2.VideoCapture("hhhhhnew.mp4")
assert cap.isOpened(), "Error opening video"

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_interval = fps * 3  # every 3 seconds

# --------------------------------
# 3. Get frame size
# --------------------------------
success, frame = cap.read()
h, w, _ = frame.shape
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# --------------------------------
# 4. Define polygon zones
# --------------------------------
zones = {
    "Zone A": Polygon([(0,0), (w//2,0), (w//2,h//3), (0,h//3)]),
    "Zone B": Polygon([(w//2,0), (w,0), (w,h//3), (w//2,h//3)]),
    "Zone C": Polygon([(0,h//3), (w,h//3), (w,2*h//3), (0,2*h//3)]),
    "Zone D": Polygon([(0,2*h//3), (w//2,2*h//3), (w//2,h), (0,h)]),
    "Zone E": Polygon([(w//2,2*h//3), (w,2*h//3), (w,h), (w//2,h)])
}

zone_colors = {
    "Zone A": (0,0,255),
    "Zone B": (255,0,0),
    "Zone C": (0,255,0),
    "Zone D": (0,255,255),
    "Zone E": (255,0,255)
}

# --------------------------------
# 5. Prepare CSV file
# --------------------------------
csv_file = "zone_c.csv"
os.makedirs("frames", exist_ok=True)

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Time_sec", "Zone A", "Zone B", "Zone C", "Zone D", "Zone E", "Frame_Image"])

# --------------------------------
# 6. Process video
# --------------------------------
frame_id = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    if frame_id % frame_interval != 0:
        frame_id += 1
        continue

    zone_counts = {z: 0 for z in zones}

    # RT-DETR inference (person only)
    results = model(frame, conf=0.2, classes=[0])
    detections = results[0]

    for box in detections.boxes:
        class_id = int(box.cls[0])
        class_name = detections.names[class_id]

        if class_name == "person":
            # Bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())

            # Draw bounding box and label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{class_name} {float(box.conf[0]):.2f}",
                        (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

            # Foot point for zone counting
            foot_x = (x1 + x2) // 2
            foot_y = y2  # bottom of bounding box
            point = Point(foot_x, foot_y)

            for zone_name, poly in zones.items():
                if poly.contains(point):
                    zone_counts[zone_name] += 1
                    cv2.circle(frame, (foot_x, foot_y), 6, zone_colors[zone_name], -1)
                    break

    # Draw zones and counts
    for zone_name, poly in zones.items():
        pts = np.array([(int(x), int(y)) for x, y in poly.exterior.coords])
        cv2.polylines(frame, [pts], True, (0,255,0), 2)
        cv2.putText(frame,
                    f"{zone_name}: {zone_counts[zone_name]}",
                    (pts[0][0] + 5, pts[0][1] + 25),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (255,255,255), 2)

    # Save frame image
    time_sec = frame_id // fps
    frame_name = f"frames/rf_frame_{time_sec}.jpg"
    cv2.imwrite(frame_name, frame)

    # Write to CSV
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            time_sec,
            zone_counts["Zone A"],
            zone_counts["Zone B"],
            zone_counts["Zone C"],
            zone_counts["Zone D"],
            zone_counts["Zone E"],
            frame_name
        ])

    print(f"Saved frame at {time_sec} sec with counts {zone_counts}")

    frame_id += 1

cap.release()
print("Processing done!")

Loading pretrain weights


TypeError: 'RFDETRBase' object is not callable